In [ ]:
from anndata import AnnData
import scanpy as sc
import squidpy as sq
import numba
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import pytometry as pm
from anndata import read_h5ad

# Comparing cell-cell interactions btw weeks

read the Anndata objects which you created in 11 pcw example code. And concatenate all.

updating the annotation names so that they overlap between weeks

In [ ]:
adata6 = read_h5ad('6week_adata.h5ad')
adata8 = read_h5ad('8_5week.h5ad')
adata8.obs['annotation'].replace({"Vim+ mesenchymal": "Vim+ mes."}, inplace=True)
adata8.obs['annotation'].replace({"Chondroblasts": "Chondroblast"}, inplace=True)
adata11 = read_h5ad('11adata.h5ad')
adata11.obs["ROI"] = "11week"
adata12 = read_h5ad('12adata.h5ad')
adata12.obs["ROI"] = "12week"
adata13 = read_h5ad('13adata.h5ad')
adata13.obs["ROI"] = "13week"
adata13.obs['annotation'].replace({"Vim+ mesenchymal": "Vim+ mes."}, inplace=True)

In [ ]:
adata_all = adata6.concatenate(adata8,adata11,adata12,adata13)

plotting all the cell-cell adjacency heatmaps all together

In [ ]:
level='annotation'
nu=0
celltype2id=dict(zip(np.unique(adata_all.obs[level]),range(0,len(np.unique(adata_all.obs[level])))))
allmat=np.zeros([len(np.unique(adata_all.obs[level])),len(np.unique(adata_all.obs[level])),len(adata_all.obs['ROI'].unique())])
for samp in adata_all.obs['ROI'].unique():
    adsu=adata_all[adata_all.obs['ROI']==samp]
    adsu.obs['annotation'] = pd.Categorical(adsu.obs['annotation'])
    sq.gr.spatial_neighbors(adsu, radius=50, coord_type="generic")
    nenrichsub=sq.gr.nhood_enrichment(adsu, cluster_key=level,copy = True)
    sq.gr.nhood_enrichment(adsu, cluster_key=level)
    #matrix=adsu.uns['annotation_nhood_enrichment']['zscore']=np.matrix(pd.DataFrame(adsu.uns['annotation_nhood_enrichment']['zscore']).fillna(0))
    nhood_sample=pd.DataFrame(nenrichsub[0],index=np.unique(adsu.obs[level]),columns=np.unique(adsu.obs[level]))
    for ct in nhood_sample.columns:
        for ct2 in nhood_sample.index:
            allmat[celltype2id[ct],celltype2id[ct2],nu]=nhood_sample.loc[ct,ct2]

    sq.pl.nhood_enrichment(adsu, cluster_key=level, method="ward",mode='zscore',vmax=100,vmin=-100,cmap='coolwarm',show=False)
    #plt.title(samp)
    #plt.savefig(general_path+'/ISS_global_analysis/'+samp+'_nhood_enrichment_'+level+'.pdf')
    nu=nu+1

fetching the dataframe from specific layers corresponding to different weeks. These datasets will be used for scaling in R (scale() function). For the simplicity we already provided the scaled datasets for the common cell types for all weeks in the data repository.

In [ ]:
layer_index = 4  # Change this to the desired layer index

# Create an empty DataFrame with appropriate row and column indices
num_rows, num_cols = allmat.shape[0], allmat.shape[1]
grouped_cells =pd.DataFrame(index=np.unique(adata_all.obs[level]),columns=np.unique(adata_all.obs[level]))

# Populate the DataFrame using the selected layer
for row_idx in range(num_rows):
    for col_idx in range(num_cols):
        grouped_cells.iloc[row_idx, col_idx] = allmat[row_idx, col_idx, layer_index]

# Print the DataFrame
print(grouped_cells)


In [ ]:
grouped_cells.to_csv('13w_interaction_enrichmentscores.csv', index=True)

changes in cell type adjacency patterns over time

creating a 3d matrix with the scaled enrichment scores (scaling done in R using scale() function)-Scaling is performed using interaction_enrichmentscores csv files created in the previous step (provided in the data repository)

In [ ]:
scaled_week6=pd.read_table('/Users/sanem.sanyar/Python_squidpy_analysis/fetallung_analysis_python/6w/scaled_week6.csv',sep=',')
scaled_week8_5=pd.read_table('/Users/sanem.sanyar/Python_squidpy_analysis/fetallung_analysis_python/6w/scaled_week8.5.csv',sep=',')
scaled_week11=pd.read_table('/Users/sanem.sanyar/Python_squidpy_analysis/fetallung_analysis_python/6w/scaled_week11.csv',sep=',')
scaled_week12=pd.read_table('/Users/sanem.sanyar/Python_squidpy_analysis/fetallung_analysis_python/6w/scaled_week12.csv',sep=',')
scaled_week13=pd.read_table('/Users/sanem.sanyar/Python_squidpy_analysis/fetallung_analysis_python/6w/scaled_week13.csv',sep=',')

In [ ]:
scaled_week6 = scaled_week6.set_index('Unnamed: 0')
scaled_week8_5 = scaled_week8_5.set_index('Unnamed: 0')
scaled_week11 = scaled_week11.set_index('Unnamed: 0')
scaled_week12 = scaled_week12.set_index('Unnamed: 0')
scaled_week13 = scaled_week13.set_index('Unnamed: 0')

In [ ]:
# Assuming the dataframes are named as follows:
# scaled_week6, scaled_week8_5, scaled_week11, scaled_week12, scaled_week13

data_matrices = [df.to_numpy() for df in [scaled_week6, scaled_week8_5, scaled_week11, scaled_week12, scaled_week13]]

In [ ]:
overlay = np.stack(data_matrices, axis=-1)

In [ ]:
import matplotlib.pyplot as plt

weeks = ['week6', 'week8.5', 'week11', 'week12', 'week13']
column_names = scaled_week6.columns  # Assuming column names are the same across all dataframes
index_names = scaled_week6.index  # Assuming row indexing is the same

# Create plots: one plot for each column
for col_idx, col_name in enumerate(column_names):
    plt.figure(figsize=(10, 6))
    for row_idx, row_name in enumerate(index_names):
        # Extract the data for this row and column across all weeks
        data_to_plot = overlay[row_idx, col_idx, :]
        plt.plot(weeks, data_to_plot, marker='o', label=f"{row_name}")

    plt.title(f"Time Series for {col_name}")
    plt.xlabel('Weeks')
    plt.ylabel('Value')
    plt.legend(title="Row", bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True)
    plt.tight_layout()
    plt.show()
    
# Create plots: one plot for each column
for col_idx, col_name in enumerate(column_names):
    plt.figure(figsize=(10, 6))
    for row_idx, row_name in enumerate(index_names):
        # Extract the data for this row and column across all weeks
        data_to_plot = overlay[row_idx, col_idx, :]
        plt.plot(weeks, data_to_plot, marker='o', label=f"{row_name}")

    plt.title(f"Time Series for {col_name}")
    plt.xlabel('Weeks')
    plt.ylabel('Value')
    plt.legend(title="Row", bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True)
    plt.tight_layout()
    
    # Save the figure as a PDF
    plt.savefig(f"{col_name}_plot.pdf")
    
    # Show the plot
    plt.show()


In [ ]:
# Assuming scaled_week6 is representative of all for column and index names
column_names = scaled_week6.columns.tolist()
index_names = scaled_week6.index.tolist()

# Comparing cell-cell adjacency of the proliferating cells across weeks

In [ ]:
level = 'proliferation'
nu = 0
celltype2id = dict(zip(np.unique(adata_all.obs[level]), range(len(np.unique(adata_all.obs[level])))))
allmat = np.zeros([len(np.unique(adata_all.obs[level])), len(np.unique(adata_all.obs[level])), len(adata_all.obs['ROI'].unique())])

for samp in adata_all.obs['ROI'].unique():
    adsu = adata_all[adata_all.obs['ROI'] == samp]
    adsu.obs['proliferation'] = pd.Categorical(adsu.obs['proliferation'])
    sq.gr.spatial_neighbors(adsu, radius=50, coord_type="generic")
    nenrichsub = sq.gr.nhood_enrichment(adsu, cluster_key=level, copy=True)
    nhood_sample = pd.DataFrame(nenrichsub[0], index=np.unique(adsu.obs[level]), columns=np.unique(adsu.obs[level]))

    for ct in nhood_sample.columns:
        for ct2 in nhood_sample.index:
            allmat[celltype2id[ct], celltype2id[ct2], nu] = nhood_sample.loc[ct, ct2]

    # Generate the plot
    ax = sq.pl.nhood_enrichment(adsu, cluster_key=level, method="ward", mode='zscore', vmax=100, vmin=-100, cmap='coolwarm', show=False)

    # Save the plot to a PDF file
    plt.title(f"{samp} nhood enrichment {level}")
    plt.savefig(f"{samp}_nhood_enrichment_{level}.pdf")
    plt.close()  # Close the figure to free up memory

    nu += 1

fetching the dataframe from specific layers corresponding to different weeks. These datasets will be used for scaling in R (scale() function). For the simplicity we already provided the scaled datasets for the common cell types for all weeks in the data repository.

In [ ]:
layer_index = 4  # Change this to the desired layer index

# Create an empty DataFrame with appropriate row and column indices
num_rows, num_cols = allmat.shape[0], allmat.shape[1]
grouped_cells =pd.DataFrame(index=np.unique(adata_all.obs[level]),columns=np.unique(adata_all.obs[level]))

# Populate the DataFrame using the selected layer
for row_idx in range(num_rows):
    for col_idx in range(num_cols):
        grouped_cells.iloc[row_idx, col_idx] = allmat[row_idx, col_idx, layer_index]

# Print the DataFrame
print(grouped_cells)


In [ ]:
grouped_cells.to_csv('13w_pro_interaction_enrichment.csv', index=True)

In [ ]:
#creating a 3d matrix with the scaled enrichment scores (scaling done in R using scale() function)-Scaling is performed using interaction_enrichmentscores csv files created in the previous step.

In [ ]:
#loading scaled dataframes

In [ ]:
pro_scaled_week6=pd.read_table('/Users/sanem.sanyar/Python_squidpy_analysis/fetallung_analysis_python/6w/pro_scaled_week6.csv',sep=',')
pro_scaled_week8_5=pd.read_table('/Users/sanem.sanyar/Python_squidpy_analysis/fetallung_analysis_python/6w/pro_scaled_week8.5.csv',sep=',')
pro_scaled_week11=pd.read_table('/Users/sanem.sanyar/Python_squidpy_analysis/fetallung_analysis_python/6w/pro_scaled_week11.csv',sep=',')
pro_scaled_week12=pd.read_table('/Users/sanem.sanyar/Python_squidpy_analysis/fetallung_analysis_python/6w/pro_scaled_week12.csv',sep=',')
pro_scaled_week13=pd.read_table('/Users/sanem.sanyar/Python_squidpy_analysis/fetallung_analysis_python/6w/pro_scaled_week13.csv',sep=',')

In [ ]:
pro_scaled_week6 = pro_scaled_week6.set_index('Unnamed: 0')
pro_scaled_week8_5 = pro_scaled_week8_5.set_index('Unnamed: 0')
pro_scaled_week11 = pro_scaled_week11.set_index('Unnamed: 0')
pro_scaled_week12 = pro_scaled_week12.set_index('Unnamed: 0')
pro_scaled_week13 = pro_scaled_week13.set_index('Unnamed: 0')

In [ ]:
# Assuming the dataframes are named as follows:
# scaled_week6, scaled_week8_5, scaled_week11, scaled_week12, scaled_week13

pro_data_matrices = [df.to_numpy() for df in [pro_scaled_week6, pro_scaled_week8_5, pro_scaled_week11, pro_scaled_week12, pro_scaled_week13]]

pro_overlay = np.stack(pro_data_matrices, axis=-1)

In [ ]:
weeks = ['week6', 'week8.5', 'week11', 'week12', 'week13']
column_names = pro_scaled_week6.columns  # Assuming column names are the same across all dataframes
index_names = pro_scaled_week6.index  # Assuming row indexing is the same

# Create plots: one plot for each column
for col_idx, col_name in enumerate(column_names):
    plt.figure(figsize=(10, 6))
    for row_idx, row_name in enumerate(index_names):
        # Extract the data for this row and column across all weeks
        data_to_plot = pro_overlay[row_idx, col_idx, :]
        plt.plot(weeks, data_to_plot, marker='o', label=f"{row_name}")

    plt.title(f"Time Series for {col_name}")
    plt.xlabel('Weeks')
    plt.ylabel('Value')
    plt.legend(title="Row", bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:
# Example category names and week names
category_names = ['ASM', 'Endothelial', 'Immune', 'Mesenchymal', 'Proliferating ASM', 'Proliferating Endothelial', 'Proliferating Immune', 'Proliferating Mesenchymal', 'Proliferating SOX2 high epit.', 'Proliferating SOX9 high epit.','SOX2 high epit.','SOX9 high epit.']  # 12 categories
week_names = ['6week', '8.5week', '11week', '12week', '13week']  # 5 weeks

# Initialize an empty DataFrame
diagonal_pro = pd.DataFrame(index=category_names, columns=week_names)

# Fill the DataFrame with diagonal values from pro_overlay
for i, cat in enumerate(category_names):
    for j, week in enumerate(week_names):
        diagonal_pro.at[cat, week] = pro_overlay[i, i, j]  # Ensure this indexing matches the actual array size

# Output the DataFrame to verify
print(diagonal_pro)

In [ ]:
# Define a custom order for the rows
custom_row_order = ['ASM', 'Proliferating ASM', 'Endothelial', 'Proliferating Endothelial', 
                'Immune', 'Proliferating Immune', 
                'Mesenchymal', 'Proliferating Mesenchymal', 'SOX2 high epit.', 
                'Proliferating SOX2 high epit.', 'SOX9 high epit.', 'Proliferating SOX9 high epit.']

# Reorder the DataFrame rows according to the custom order
diagonal_pro_reordered = diagonal_pro.loc[custom_row_order]



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind


# Calculating p-values
p_values = {}
for i in range(0, len(diagonal_pro_reordered.index), 2):
    non_prolif = diagonal_pro_reordered.iloc[i]
    prolif = diagonal_pro_reordered.iloc[i+1]
    _, p_val = ttest_ind(non_prolif, prolif)
    p_values[diagonal_pro_reordered.index[i]] = p_val

# Transposing the DataFrame for plotting
transposed_data = diagonal_pro_reordered.transpose()

# Adjust matplotlib settings locally if needed
plt.rcParams.update({'figure.figsize': (30, 15), 'figure.dpi': 200})

# Create a larger box plot explicitly specifying size
fig, ax = plt.subplots(figsize=(25, 10))  # This should ensure that figsize is respected
transposed_data = diagonal_pro_reordered.transpose()
transposed_data.plot(kind='box', ax=ax)

# Annotating the box plot with p-values
for line, p_val in zip(range(0, len(p_values) * 2, 2), p_values.values()):
    x = line + 0.5
    y = max(transposed_data.iloc[:, line].max(), transposed_data.iloc[:, line+1].max()) + 0.1
    ax.text(x, y, f'p={p_val:.2e}', horizontalalignment='center', color='red', fontsize=12)

ax.set_title('Cell type enrichment with itself', fontsize=16)
ax.set_xlabel('Cell Types', fontsize=14)
ax.set_ylabel('Expression Level', fontsize=14)
ax.grid(True)

# Save the plot as a PDF and show it
plt.savefig('cell_type_enrichment.pdf', format='pdf', bbox_inches='tight')
plt.show()

plt.ion()  # Re-enable interactive mode if you prefer it for other parts of your notebook

In [ ]:
results_df = pd.DataFrame(index=diagonal_pro_reordered.index[::2], columns=diagonal_pro_reordered.columns)  # Every other index is a standard type

# Fill the new DataFrame by dividing the standard by the proliferating type
for standard, proliferating in zip(diagonal_pro_reordered.index[::2], diagonal_pro_reordered.index[1::2]):
    results_df.loc[standard] = diagonal_pro_reordered.loc[standard] / diagonal_pro_reordered.loc[proliferating]

# Output the new DataFrame to verify
print(results_df)

In [ ]:
ax = results_df.T.plot(figsize=(10, 8), marker='o')  # Transpose df to get weeks on the x-axis
plt.title('Cell Type Ratios Over Weeks')
plt.xlabel('Weeks')
plt.ylabel('Ratio')
plt.grid(True)
plt.legend(title='Cell Type')
plt.savefig('nonprof_prof_ratio_interaction.pdf', format='pdf', bbox_inches='tight')
plt.show()